In [1]:
! pip install azure-storage-blob requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.5 MB/s eta 0:00:0000:01


In [20]:
import os
import json
import hashlib
from datetime import datetime, timedelta
from tqdm import tqdm
import requests as req
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, explode, col, array, lit, transform
from azure.storage.blob import BlobServiceClient
from hdfs import InsecureClient as HdfsClient
from io import BytesIO
import os
import json
import hashlib
from io import BytesIO
from datetime import datetime, timedelta
from PIL import Image
from tqdm import tqdm
from azure.storage.filedatalake import DataLakeServiceClient
import requests as req
from PIL import Image
import requests as req
from pyspark.sql import SparkSession
from azure.storage.filedatalake import DataLakeServiceClient, FileSystemClient
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [5]:
connection_string = os.getenv('AZURE_CONNECTION_STRING')
storage_account_name = 'bdmprojectaccount'
storage_container_name = 'bdmcontainerp1'
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
spark = SparkSession.builder \
    .appName("BDMProject") \
    .getOrCreate()
spark.sparkContext.setLogLevel("INFO")

25/05/14 19:27:20 WARN Utils: Your hostname, Medio-Metro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
25/05/14 19:27:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/14 19:27:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/14 19:27:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
from azure.storage.filedatalake import DataLakeServiceClient

datalake_client = DataLakeServiceClient.from_connection_string(connection_string)
file_system_client = datalake_client.get_file_system_client(storage_container_name)
directory_client = file_system_client.get_directory_client("landing_zone/accommodation/Barcelona/")
file_client = directory_client.get_file_client("2025-03-24_2025-03-25.json")

download = file_client.download_file()
contents = download.readall()


# Transportation

Find out more in: https://openrouteservice.org/dev/#/api-docs/optimization/post
Alternatives: https://github.com/graphhopper/graphhopper/blob/master/README.md#Map-Matching
https://github.com/VROOM-Project/vroom/blob/master/docs/API.md

In [11]:
from hdfs import InsecureClient

def test_spark_connection() -> bool:
    """
    Verifies the Spark session is active by performing a simple operation.
    Returns True if the session is responsive, False otherwise.
    """
    try:
        # Simple operation: create a small DataFrame and collect
        count = spark.range(0, 1).count()
        print (count == 1)
    except Exception:
        return print(False)

def test_hdfs_connection():
    try:
        hdfs_url = os.environ['HDFS_URL']
        client = InsecureClient(hdfs_url, user='hdfs')  # use appropriate user

        # List root directory
        print("Listing root directory:")
        print(client.list('/'))

        # Write a test file
        test_path = '/tmp/test_hdfs_connection.txt'
        test_content = 'HDFS connection successful!'
        client.write(test_path, data=test_content, overwrite=True)
        print(f"File written to {test_path}")

        # Read back the file
        with client.read(test_path, encoding='utf-8') as reader:
            content = reader.read()
            print("File content:")
            print(content)

        return True
    except Exception as e:
        print(f"❌ HDFS connection failed: {e}")
        return False

# Run test
if __name__ == '__main__':
    test_hdfs_connection()
    test_spark_connection()

Listing root directory:
['tmp']
File written to /tmp/test_hdfs_connection.txt
File content:
HDFS connection successful!
True


In [9]:
destination_ids = {
    "Barcelona": "-372490",
    "Rome": "-126693",
    "Madrid": "-390625",
    "Paris": "-1456928"
}

destination_coords = {
    'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
    'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
    'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
    'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
}

accommodation_endpoint = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
weather_endpoint = 'https://archive-api.open-meteo.com/v1/archive'

headers = {
    "x-rapidapi-key": os.environ["RAPID_API_KEY"],
    "x-rapidapi-host": os.environ["RAPID_API_HOST"]
}

accommodation_query = {
    "dest_id": '',
    "search_type": "CITY",
    "arrival_date": '',
    "departure_date": '',
    "adults": "2",
    "children_age": "0",
    "room_qty": "1",
    "page_number": "1",
    "units": "metric",
    "temperature_unit": "c",
    "languagecode": "en-us",
    "currency_code": "EUR"
}

weather_metrics = 'temperature_2m,rain,snowfall,precipitation,cloud_cover,wind_speed_10m,sunshine_duration'

weather_query = {
    'latitude': '',
    'longitude': '',
    'hourly': weather_metrics,
    'start_date': '',
    'end_date': ''
}

In [43]:
def build_struct(flat_schema: dict) -> StructType:
    fields = []
    for name, typ in flat_schema.items():
        if isinstance(typ, list):
            elem = typ[0]
            if isinstance(elem, dict):
                struct = build_struct(elem)
                fields.append(StructField(name, ArrayType(struct), True))
            else:
                spark_type = {'int': IntegerType(), 'float': DoubleType(), 'str': StringType(), 'bool': BooleanType()}.get(elem, StringType())
                fields.append(StructField(name, ArrayType(spark_type), True))
        else:
            spark_type = {'int': IntegerType(), 'float': DoubleType(), 'str': StringType(), 'bool': BooleanType()}.get(typ, StringType())
            fields.append(StructField(name, spark_type, True))
    return StructType(fields)

def string_to_sha256(text: str) -> str:
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def flatten_df(df):
    from pyspark.sql.types import StructType
    flat_cols = []
    nested_cols = []
    for field in df.schema.fields:
        if isinstance(field.dataType, StructType):
            nested_cols.append(field.name)
        else:
            flat_cols.append(col(field.name))
    for nested in nested_cols:
        for f in df.schema[nested].dataType.fields:
            flat_cols.append(col(f"{nested}.{f.name}").alias(f"{nested}_{f.name}"))
    return df.select(flat_cols)

# Configuración del Data Lake
# Asume que 'file_system_name' es el filesystem de Delta Lake
def get_data_lake_service(account_url: str, credential) -> DataLakeServiceClient:
    return DataLakeServiceClient(account_url=account_url, credential=credential)

@udf(StringType())
def sha256_udf(url: str) -> str:
    return hashlib.sha256(url.encode('utf-8')).hexdigest() if url else None

@udf(StringType())
def standardized_hours_udf(ts: str) -> str:
    return datetime.fromisoformat(ts).strftime('%H:%M') if ts else None


def file_exists(fs_client, path: str) -> bool:
    try:
        file_client = fs_client.get_file_client(path)
        return file_client.exists()
    except Exception:
        return False


def upload_file(fs_client, path: str, data: bytes, overwrite=True) -> None:
    file_client = fs_client.get_file_client(path)
    if overwrite and file_client.exists():
        file_client.delete_file()
    file_client.create_file()
    file_client.append_data(data, offset=0)
    file_client.flush_data(len(data))

# ---------------------------------------------------------------------
# FLATTEN, CAST, SCHEMA Y COMPRESIÓN DE IMÁGENES (sin cambios)
# ---------------------------------------------------------------------


def load_json_schema(path: str) -> dict:
    with open(path) as f:
        return json.load(f)

def compress_image(image_bytes: bytes, max_width: int = 1024, quality: int = 75) -> bytes:
    with Image.open(BytesIO(image_bytes)) as img:
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        if img.width > max_width:
            ratio = max_width / float(img.width)
            new_height = int(img.height * ratio)
            img = img.resize((max_width, new_height), Image.LANCZOS)
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality, optimize=True)
        return buffer.getvalue()

def process_accommodation_record(record: dict, schema: dict) -> dict:
    # Crear DF inferido
    accommodation_schema_dict = json.loads(open('accommodation_schema.json').read())
    accommodation_struct = build_struct(accommodation_schema_dict)
     # Crear DF completo e inferido
    df = spark.read.json(spark.sparkContext.parallelize([json.dumps(record)]))

    # Extraer columnas planas y hasta 3 niveles de anidación

    all_cols = []
    for field in df.schema.fields:
        if isinstance(field.dataType, StructType):
            for subfield in field.dataType.fields:
                if isinstance(subfield.dataType, StructType):
                    for subsubfield in subfield.dataType.fields:
                        if isinstance(subsubfield.dataType, StructType):
                            for subsubsubfield in subsubfield.dataType.fields:
                                all_cols.append(
                                    col(f"{field.name}.{subfield.name}.{subsubfield.name}.{subsubsubfield.name}").alias(f"{field.name}_{subfield.name}_{subsubfield.name}_{subsubsubfield.name}")
                                )
                        else:
                            all_cols.append(
                                col(f"{field.name}.{subfield.name}.{subsubfield.name}").alias(f"{field.name}_{subfield.name}_{subsubfield.name}")
                            )
                else:
                    all_cols.append(
                        col(f"{field.name}.{subfield.name}").alias(f"{field.name}_{subfield.name}")
                    )
        else:
            all_cols.append(col(field.name))

    df_flat = df.select(*all_cols)

    # Generar hashes de fotos en Python
    photo_urls = record.get("property", {}).get("photoUrls", [])
    photo_hashes = [string_to_sha256(u) for u in photo_urls]
    df_final = df_flat.withColumn("property_photoHash", lit(photo_hashes))

    # Serializar y validar con esquema oficial
    json_flat = df_final.toJSON().first()
    df_valid = spark.read.schema(accommodation_struct).json(spark.sparkContext.parallelize([json_flat]))
    return df_valid.collect()[0].asDict()

# --- Procesar registro de clima con Spark flatten y luego aplicar esquema ---
def process_weather_record(raw: dict, schema: dict) -> dict:
    weather_schema_dict = json.loads(open('weather_schema.json').read())
    weather_struct = build_struct(weather_schema_dict)
    raw_hourly = raw.get('hourly', {})
    raw_hourly['time'] = [datetime.fromisoformat(t).strftime('%H:%M') for t in raw_hourly.get('time', [])]

    # Crear DataFrame inferido a partir del JSON anidado
    df = spark.read.json(spark.sparkContext.parallelize([json.dumps({'hourly': raw_hourly})]))
    # Flatten structure 'hourly'
    df_flat = flatten_df(df.select('hourly')).withColumnRenamed('hourly_time', 'time')

    # Serializar a JSON y validar con esquema oficial
    json_flat = df_flat.toJSON().first()
    df_valid = spark.read.schema(weather_struct).json(spark.sparkContext.parallelize([json_flat]))
    return df_valid.collect()[0].asDict()

# --- Procesado de imágenes sin cambios ---
def process_accommodation_images(photo_urls: list, fs_client, city: str) -> None:
    for url in photo_urls:
        sha = string_to_sha256(url)
        path = f"landing_zone/accommodation_images/{city}/{sha}.jpg"
        trusted_path = f"trusted_zone/accommodation_images/{city}/{sha}.jpg"
        if file_exists(fs_client, path) and file_exists(fs_client, trusted_path):
            continue
        if file_exists(fs_client, path):
            data = fs_client.get_file_client(path).download_file().readall()
        else:
            res = req.get(url, stream=True)
            try:
                res.raise_for_status()
            except:
                continue
            data = res.content
            upload_file(fs_client, path, data)
        compressed = compress_image(data, max_width=800, quality=70)
        upload_file(fs_client, trusted_path, compressed)


def get_and_sync_accommodation(
    fs,
    start: datetime,
    end: datetime,
    cities: dict,
    query_template: dict,
    schema_file: str
):
    schema = load_json_schema(schema_file)
    delta = timedelta(days=1)

    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        arrival = single_date.strftime('%Y-%m-%d')
        departure = (single_date + delta).strftime('%Y-%m-%d')
        for city, dest_id in cities.items():
            landing_path = f"landing_zone/accommodation/{city}/{arrival}_{departure}.json"
            trusted_path = f"trusted_zone/accommodation/{city}/{arrival}_{departure}.json"

            # filtro: si existe en landing y trusted, nada que hacer
            if file_exists(fs, landing_path) and file_exists(fs, trusted_path):
                continue

            # obtener JSON: de landing o API
            if file_exists(fs, landing_path):
                raw = fs.get_file_client(landing_path).download_file().readall().decode('utf-8')
                data = json.loads(raw)
            else:
                params = dict(query_template, dest_id=dest_id, arrival_date=arrival, departure_date=departure)
                res = req.get(accommodation_endpoint, params=params)
                res.raise_for_status()
                data = res.json()
                upload_file(fs, landing_path, json.dumps(data).encode('utf-8'))

            # procesar imágenes
            photo_urls = [u for h in data['data']['hotels'] for u in h['property']['photoUrls']]
            process_accommodation_images(photo_urls, fs, city)

            # transformar y guardar registros en zona confiable
            docs = [process_accommodation_record(r, schema) for r in data['data']['hotels']]
            upload_file(fs, trusted_path, json.dumps(docs).encode('utf-8'))


def get_and_sync_weather(
    fs,
    start: datetime,
    end: datetime,
    coords: dict,
    query_template: dict,
    schema_file: str
):
    schema = load_json_schema(schema_file)
    delta = timedelta(days=1)

    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        prev_start = (single_date - timedelta(days=365)).strftime('%Y-%m-%d')
        for city, coord in coords.items():
            landing_path = f"landing_zone/weather/{city}/{prev_start}.json"
            trusted_path = f"trusted_zone/landing_zone/weather/{city}/{prev_start}.json"

            # filtro: si existe en landing y trusted, nada que hacer
            if file_exists(fs, landing_path) and file_exists(fs, trusted_path):
                continue

            # obtener JSON: de landing o API
            if file_exists(fs, landing_path):
                raw = fs.get_file_client(landing_path).download_file().readall().decode('utf-8')
                data = json.loads(raw)
            else:
                params = dict(query_template, latitude=coord['latitude'], longitude=coord['longitude'], start_date=prev_start, end_date=prev_start)
                res = req.get(weather_endpoint, params=params)
                res.raise_for_status()
                data = res.json()
                upload_file(fs, landing_path, json.dumps(data).encode('utf-8'))

            # transformar y guardar en zona confiable
            doc = process_weather_record(data, schema)
            upload_file(fs, trusted_path, json.dumps(doc).encode('utf-8'))

def list_files(fs_client, prefix: str) -> list:
    return [p.name for p in fs_client.get_paths(path=prefix) if not p.is_directory]

def find_missing_blobs(
    fs,
    landing_prefix: str
) -> list:
    missing = []
    for path in list_files(fs, landing_prefix):
        trusted_path = f"trusted_zone/{path}"
        if not file_exists(fs, trusted_path):
            missing.append(path)
    return missing


def backfill_missing(
    fs,
    landing_prefix: str,
    schema_file: str = None
) -> None:
    schema = load_json_schema(schema_file) if schema_file else None
    to_fill = find_missing_blobs(fs, landing_prefix)

    for landing_path in tqdm(to_fill):
        trusted_path = f"trusted_zone/{landing_path}"
        data_bytes = fs.get_file_client(landing_path).download_file().readall()

        # imágenes de alojamiento
        if landing_path.startswith('landing_zone/accommodation_images/'):
            compressed = compress_image(data_bytes, max_width=800, quality=70)
            upload_file(fs, trusted_path, compressed)
            continue

        # JSON de alojamiento
        if landing_path.startswith('landin_zone/accommodation/'):
            data = json.loads(data_bytes.decode('utf-8'))
            docs = [process_accommodation_record(r, schema) for r in data['data']['hotels']]
            upload_file(fs, trusted_path, json.dumps(docs).encode('utf-8'))
            continue

        # JSON de clima
        if landing_path.startswith('landing_zone/weather/'):
            data = json.loads(data_bytes.decode('utf-8'))
            doc = process_weather_record(data, schema)
            upload_file(fs, trusted_path, json.dumps(doc).encode('utf-8'))
        break



In [44]:
# Initialize DataLakeServiceClient once
datalake_client = DataLakeServiceClient.from_connection_string(connection_string)
file_system_client = datalake_client.get_file_system_client(storage_container_name)
hdfs_service = HdfsClient(os.environ['HDFS_URL'])


# parameters
start = datetime.strptime('2025-06-21', '%Y-%m-%d')
end = datetime.strptime('2025-06-21', '%Y-%m-%d')
cities = list(destination_ids.keys())

# sync on the fly
get_and_sync_accommodation(file_system_client, start, end, destination_ids, accommodation_query, 'accommodation_schema.json')
get_and_sync_weather(file_system_client, start, end, destination_coords, weather_query, 'weather_schema.json')

  0%|          | 0/1 [00:00<?, ?it/s]25/05/14 20:25:49 INFO BlockManagerInfo: Removed broadcast_2347_piece0 on 192.168.1.35:63234 in memory (size: 14.6 KiB, free: 434.3 MiB)
25/05/14 20:25:49 INFO BlockManagerInfo: Removed broadcast_2346_piece0 on 192.168.1.35:63234 in memory (size: 45.9 KiB, free: 434.4 MiB)
25/05/14 20:25:49 INFO SparkContext: Starting job: json at <unknown>:0
25/05/14 20:25:49 INFO DAGScheduler: Got job 2348 (json at <unknown>:0) with 8 output partitions
25/05/14 20:25:49 INFO DAGScheduler: Final stage: ResultStage 2348 (json at <unknown>:0)
25/05/14 20:25:49 INFO DAGScheduler: Parents of final stage: List()
25/05/14 20:25:49 INFO DAGScheduler: Missing parents: List()
25/05/14 20:25:49 INFO DAGScheduler: Submitting ResultStage 2348 (MapPartitionsRDD[10864] at json at <unknown>:0), which has no missing parents
25/05/14 20:25:49 INFO MemoryStore: Block broadcast_2348 stored as values in memory (estimated size 20.6 KiB, free 434.2 MiB)
25/05/14 20:25:49 INFO MemoryStor

25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2442_piece0 on 192.168.1.35:63234 in memory (size: 14.7 KiB, free: 434.2 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2444_piece0 on 192.168.1.35:63234 in memory (size: 45.1 KiB, free: 434.2 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2440_piece0 on 192.168.1.35:63234 in memory (size: 41.5 KiB, free: 434.2 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2445_piece0 on 192.168.1.35:63234 in memory (size: 45.1 KiB, free: 434.3 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2441_piece0 on 192.168.1.35:63234 in memory (size: 41.5 KiB, free: 434.3 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2443_piece0 on 192.168.1.35:63234 in memory (size: 10.1 KiB, free: 434.3 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: Removed broadcast_2446_piece0 on 192.168.1.35:63234 in memory (size: 45.1 KiB, free: 434.4 MiB)
25/05/14 20:27:23 INFO BlockManagerInfo: 

In [ ]:
backfill_missing(file_system_client, 'landing_zone/accommodation/', 'accommodation_schema.json')
backfill_missing(file_system_client, 'landing_zone/accommodation_images/',)
backfill_missing(file_system_client, 'landing_zone/weather/', 'weather_schema.json')

  2%|▏         | 37/1890 [00:24<19:17,  1.60it/s]